In [1]:
import pandas as pd
df = pd.read_csv("./data/interviews/interviews_en.csv")
df_class = pd.read_csv('data/target.csv', index_col=0)
df['Class'] = df_class['Class']

In [2]:
df.head(3)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,file_name,label,text,Manually_Labeled,text_bg,Class
0,0,0,0,BG_Box_AleksandarPovetkin_vs_AnthonyDjoshua_NO,NO,"I am in very good shape, I had an excellent tr...",True,"Аз съм в много добра форма, проведох отличен т...",0.0
1,1,1,1,BG_Box_AndiRuiz_vs_AnthonyDjoshua_YES,YES,"Everyone underestimates me, but it will be a t...",True,"Всички ме подценяват, но това ще бъде страшна ...",1.0
2,2,2,2,BG_Box_AnthonyDjoshua_vs_AleksandarPovetkin_YES,YES,"Leaving aside the technique and qualities, we ...",True,"Ако оставим настрана техниката и качествата, и...",1.0


In [3]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.text, df.Class, random_state=42)
print(X_train.shape)
print(X_test.shape)

(37,)
(13,)


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df =1, stop_words='english' ,token_pattern=r'\b[a-zA-Z]+\b')

In [5]:
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm

<37x1020 sparse matrix of type '<class 'numpy.int64'>'
	with 1925 stored elements in Compressed Sparse Row format>

In [6]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression(solver='liblinear')
logReg.fit(X_train_dtm,y_train)

LogisticRegression(solver='liblinear')

In [7]:
X_test_dtm = vect.transform(X_test)
X_test_dtm

<13x1020 sparse matrix of type '<class 'numpy.int64'>'
	with 341 stored elements in Compressed Sparse Row format>

In [8]:
X_test_dtm

<13x1020 sparse matrix of type '<class 'numpy.int64'>'
	with 341 stored elements in Compressed Sparse Row format>

In [9]:
from sklearn import metrics
y_pred_class = logReg.predict(X_test_dtm)
metrics.accuracy_score(y_test, y_pred_class)

0.6153846153846154

In [29]:
y_pred_class

array([0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.])

In [31]:
df.text[0]

"I am in very good shape, I had an excellent training camp.\r\nJoshua is one of the strongest heavyweight boxers in the world.\r\nThat's why I'm happy to have the opportunity to be in the ring against him and I want to give the fans a nice match.\r\nThis meeting will show everything, Anthony is a strong boxer, but I am also strong.\r\nWhen I fought with Klitschko, I was weaker and in worse shape than I am now."

if we get lucky, we can get 61% but it really depends on the split luck

In [22]:
type(X_train)

pandas.core.series.Series

In [13]:
#run prediction on this
X_sample = X_train.loc[0]
X_sample 

"I am in very good shape, I had an excellent training camp.\r\nJoshua is one of the strongest heavyweight boxers in the world.\r\nThat's why I'm happy to have the opportunity to be in the ring against him and I want to give the fans a nice match.\r\nThis meeting will show everything, Anthony is a strong boxer, but I am also strong.\r\nWhen I fought with Klitschko, I was weaker and in worse shape than I am now."

In [17]:
X_sample = "My name is Pierre"

In [26]:
X_sample_dtm = vect.transform([X_sample])

In [27]:
result = logReg.predict(X_sample_dtm)
result[0]

1.0

## Part 2: Using a `Pipeline` for proper cross-validation

#### with CountVectorizer()

In [129]:
# define X and y
X = df.text
y = df.Class

In [130]:
# replace the regex pattern that is used for tokenization
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df = 1,token_pattern=r"'([a-z ]+)'")

In [131]:
# import and instantiate Multinomial Naive Bayes (with the default parameters)
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [132]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('vect', vect),
    ('nb', nb),
])

In [133]:
# cross-validate the entire pipeline
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.54

#### with TfidfVectorizer

In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [120]:
vect = TfidfVectorizer(min_df = 1,stop_words='english',token_pattern=r"'([a-z ]+)'")

In [121]:
# import and instantiate Multinomial Naive Bayes (with the default parameters)
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [122]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('vect', vect),
    ('nb', nb),
])

In [123]:
# cross-validate the entire pipeline
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.54

## Part 3: Text feature selection on whole dataset with countvecotrizer

In [135]:
vect = CountVectorizer(min_df = 1,token_pattern=r"'([a-z ]+)'")

In [136]:
vect.fit_transform(X_train)

<37x14 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [137]:
vect.transform(X_test)

<13x14 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_sc = scaler.fit_transform(X)

pca = PCA()
X_pca = pca.fit_transform(X_sc)
X_pca.shape